<a href="https://colab.research.google.com/github/NikolayWTF/Building_a_GPT/blob/main/NLP_%D0%94%D0%97_3_%D0%93%D1%80%D0%B8%D0%B3%D0%BE%D1%80%D1%8C%D0%B5%D0%B2_%D0%9D%D0%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Буду решать задачу классификации тональности текст.


Для дообучения буду использовать датасет с рецензиями на фильмы

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import numpy as np
import evaluate

# Fine-tuning

## Загрузка датасета
Загружает датасет IMDb, который содержит 25,000 положительных и 25,000 отрицательных отзывов (разделены на train и test).

In [ ]:
dataset = load_dataset("imdb")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

## Загрузка модели
model_name — имя модели на Hugging Face Hub.

AutoTokenizer.from_pretrained(...) — загружает соответствующий токенизатор.

AutoModelForSequenceClassification(...) — загружает модель с "головой" для классификации. Параметр num_labels=2 указывает, что у нас бинарная классификация (0 = негатив, 1 = позитив).

In [ ]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Токенизация данных
Функция tokenize_function преобразует текст в формат, понятный модели.

padding="max_length" — дополняет каждый текст до одинаковой длины.

truncation=True — обрезает слишком длинные тексты.

map(..., batched=True) — применяет токенизацию сразу к батчам примеров (ускоряет обработку).

In [ ]:
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
# даляем колонку "text", чтобы остались только input_ids, attention_mask, label.
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
# переводим данные в формат PyTorch tensors.
tokenized_datasets.set_format("torch")

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

## Train/Test split

In [ ]:
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(2000))
eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))


## Метрики для оценки
evaluate.load("accuracy") — загружает функцию метрики.

compute_metrics — Hugging Face Trainer будет использовать эту функцию после каждой эпохи.

np.argmax — выбирает метку класса с максимальной вероятностью (логитом).

In [ ]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

## Настройки обучения
output_dir — куда сохраняется модель.

evaluation_strategy="epoch" — запуск валидации после каждой эпохи.

batch_size=8 — размер батча (можно увеличить на GPU).

num_train_epochs=2 — количество эпох.

weight_decay — регуляризация.

logging_dir — путь для логов.

logging_steps — как часто логгировать в ходе обучения.

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    eval_steps=20,
    logging_steps=20,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
)

## Trainer API

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

## Обучение

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
20,0.090100,1.635234,0.875000
40,0.104900,1.579808,0.858000
60,0.054800,1.234469,0.864000
80,0.000900,1.591497,0.831000
100,0.151900,0.965338,0.868000
120,0.001100,1.558191,0.848000
140,0.094900,1.038625,0.858000
160,0.023800,1.114524,0.869000
180,0.056400,1.087636,0.862000
200,0.107400,0.904814,0.858000


TrainOutput(global_step=500, training_loss=0.03752116254688008, metrics={'train_runtime': 550.3927, 'train_samples_per_second': 7.268, 'train_steps_per_second': 0.908, 'total_flos': 529869594624000.0, 'train_loss': 0.03752116254688008, 'epoch': 2.0})

In [ ]:
trainer.save_model("./results")

In [ ]:
tokenizer.save_pretrained("./results")

('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/vocab.txt',
 './results/added_tokens.json',
 './results/tokenizer.json')

# Тестирование fine-tune модели

In [ ]:
from transformers import pipeline

# Загружаем fine-tuned модель из директории, куда она была сохранена
model_path = "./results"

# Создаем pipeline для классификации
classifier = pipeline("text-classification", model=model_path)

# Примеры для теста
examples = [
    "This movie was fantastic! I loved the story and the characters.",
    "Terrible movie. It was boring and too long.",
    "It was okay, not the best I've seen but not the worst either."
]

# Предсказания
for text in examples:
    result = classifier(text)[0]
    label = result['label']
    score = result['score']
    print(f"Text: {text}\nPrediction: {label}, Confidence: {score:.2f}\n")


Device set to use cuda:0


Text: This movie was fantastic! I loved the story and the characters.
Prediction: LABEL_1, Confidence: 1.00

Text: Terrible movie. It was boring and too long.
Prediction: LABEL_0, Confidence: 1.00

Text: It was okay, not the best I've seen but not the worst either.
Prediction: LABEL_0, Confidence: 1.00

